## Here we'll try to replicate the results of Venturelli et al, measuring butyrate production in synthetically constructed _in vitro_ communities. We'll hope to see that predictions of butyrate production from MICOM constructed community models accurately reflect the measured butyrate production from the experiments. 

In [1]:
import os
import pandas as pd
from plotnine import *
from tqdm import tqdm
import sklearn
import numpy as np
import scipy
import micom
import micom.measures
import warnings
warnings.simplefilter(action='ignore')
%matplotlib inline

## Function for building our taxonomies, which accepts an abundance dataframe of taxonomic fractions measured in the experiments. 

In [2]:
def build_table(taxa):
    #first, define an empty taxonomy dataframe, which we will use to build models
    taxonomy = pd.DataFrame()
    #we'll iterate through the taxonomic data provided, and append individual taxonomic abundances to our taxonomy table
    for x in tqdm(taxa.index): 
        #find the row corresponding to the sample of interest in the abundance dataframe
        sample = taxa.loc[x].index.to_list()
        #isolate the genera abbreviations, and put into a list
        names = ",".join([x[:2] for x in sample]).split(',')
        #now we map the abbreviations to the file list (see next cell) to find the GSMs for each 
        file = list(map(genera.get,names))
        #now we'll put together a pandas Series to concatenate with the taxonomy table
        sample_taxa = pd.DataFrame({ 
            "id": names, #genus abbreviations 
            "abundance":taxa.loc[x], #genus abundances
            "file": file, #genus JSON file
            "sample_id":x}) #sample ID
        sample_taxa = sample_taxa.dropna(how = 'any') #drop any rows with empty values
        taxonomy = pd.concat([taxonomy,sample_taxa]) #concate the series onto the taxonomy table
    return taxonomy

## Let's start by initializing our dictionary of genera abbreviations, since the taxa are given by two letter abbreviations in the abundance dataframe. The taxa in the experiement are given at the species level, but for consistency we will build models at the genus level. Therefore, some duplicates will be seen in this dict

In [3]:
genera = {'PC':'Prevotella.json',
          'PJ':'Parabacteroides.json',
          'BV':'Bacteroides.json',
          'BF':'Bacteroides.json',
          'BO':'Bacteroides.json',
          'BT':'Bacteroides.json',
          'BC':'Bacteroides.json',
          'BY':'Bacteroides.json',
          'BU':'Bacteroides.json',
          'DP':'Desulfovibrio.json',
          'BL':'Bifidobacterium.json',
          'BA':'Bifidobacterium.json',
          'BP':'Bifidobacterium.json',
          'CA':'Collinsella.json',
          'EL':'Eggerthella.json',
          'FP':'Faecalibacterium.json',
          'CH':'Clostridium.json',
          'AC':'Anaerostipes.json',
          'BH':'Blautia.json',
          'CG':'Clostridium.json',
          'ER':'Eubacterium.json',
          'RI':'Roseburia.json',
          'CC':'Coprococcus.json',
          'DL':'Dorea.json',
          'DF':'Dorea.json'}

## Let's pull up the primary dataframe with abundances, ODs, and SCFAs. This can be found at https://github.com/RyanLincolnClark/DesignSyntheticGutMicrobiomeAssemblyFunction/commonfiles


In [4]:
os.chdir('../data')
#read in the dataframe
main = pd.read_csv('abundanceDF.csv',index_col=0) #get the dataframe
#drop HB - abbreviation not given in manuscript
main = main.drop(columns = 'HB')
#for columns 11:37, which represent binary presence/absence as a 1 or 0 for each species, we fill in zeros for NAs. 
main[main.columns[11:37]] = main[main.columns[11:37]].fillna(0).astype('int')
#we can calculate richness of each sample by summing these presence/absence numbers
main['richness'] = main[main.columns[11:37]].sum(axis = 1)
#also isolate the plate, column, and run ID for each row. These will serve as sample IDs for each individual sample
main['Plate'] = main['Plate'].astype('str').str.split('.').str[0].apply(lambda x: x.zfill(2))
main['Column'] = main['Column'].astype('str').str.split('.').str[0].apply(lambda x: x.zfill(2))
main['Run'] = main['Sequencing Run'].str[-3:]
#designate the sample ID
main['sample_id'] = 'P'+main['Plate']+main['Row']+main['Column']+'_'+main['Run']
#remove any samples that have contamination
main = main[main['Contamination?']=='No']
#set the index as the sample ID
main.set_index('sample_id',inplace = True)
main = main[~main.index.duplicated(keep = 'last')]

## Lets also pull up the componentized diet dataframe

In [5]:
os.chdir('../media')
#load the components of the medium
medium = pd.read_excel('DM38_components.xlsx')
# rename the columns of the medium for simplicity
medium = medium.rename(columns={'Component':'metabolite','Concentration (mM)':'flux'}) 
#for use in MICOM, we need to add a prefix and suffix to each entry in the reaction column
medium['reaction'] = 'EX_' + medium['metabolite'] + '_m' #reaction column
medium

,metabolite,flux,reaction
0,cl,4.300000,EX_cl_m
1,ca2,1.290106,EX_ca2_m
2,cu2,0.010000,EX_cu2_m
3,so4,10.000000,EX_so4_m
4,pydx,0.009822,EX_pydx_m
...,...,...,...
57,hco3,47.615080,EX_hco3_m
58,arab_L,21.314860,EX_arab_L_m
59,glc_D,24.978352,EX_glc_D_m
60,lac_L,28.308171,EX_lac_L_m


## Now we'll start with the low richness (1-5 species) communities

In [6]:
#isolate table to low richness
low = main[(main['richness'].astype('int') <=5)] 
#now let's isolate those columns that have fractional abundance of taxa
taxa_low = [cols for cols in low.columns if 'Fraction' in cols] #df with the taxa present in each community
#drop B.cereus - not included in paper
taxa_low = low[taxa_low].drop(columns = ['B.cereus Fraction'])
#isolate the optical density and measured butyrate of each sample and convert to a dict for future use
OD_low = low['OD'].to_dict()
meas_but_low = low['Butyrate'].to_dict()

### Build the models for low richness communities

In [7]:
os.chdir('../agora/data')
#use the build table function to construct a taxonomy table
taxonomy = build_table(taxa_low) 
#build the models from the taxonomy table with a cutoff of 0.001
manifest_low = micom.workflows.build(taxonomy, out_folder='../../models/invitro/low_richness',
                                  model_db = None, cutoff=0.001, threads=10)

#model_db equals none since we have included the file path for each GSM in the taxonomy table

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 882/882 [00:01<00:00, 503.34it/s]


[15:01:42] WARNING  Found existing models for 651 samples. Will skip those. Delete the output folder if ]8;id=841213;file:///users/nbohmann/.local/lib/python3.8/site-packages/micom/workflows/build.py\build.py]8;;\:]8;id=60249;file:///users/nbohmann/.local/lib/python3.8/site-packages/micom/workflows/build.py#95\95]8;;\
                    you would like me to rebuild them.                                                             

Output()

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Running ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━  89% 0:06:04

### Now that they are built, let's grow the low richness models with our DM38 medium

In [8]:
os.chdir('../models')
#Read the manifest produced above
manifest_low = pd.read_csv('invitro/low_richness/manifest.csv')
#fix the medium such that all samples can grow with a community growth rate of 0.3
medium_fixed = micom.workflows.fix_medium(manifest_low,'invitro/low_richness', medium, community_growth = 0.3, summarize = True)
#grow the models, using the medium and a tradeoff value of 0.7
growth = micom.workflows.grow(manifest_low,'invitro/low_richness',medium_fixed,
                           tradeoff = 0.7, threads = 10)
#isolate exchanges, and multiply the fluxes by abundances for absolute production
exchanges = micom.measures.production_rates(growth)
but_low = exchanges[exchanges.reaction.str.startswith('EX_but(e)')].rename(columns = {'flux':'predicted'})
#map the optical density of each sample onto the dataframe
but_low['OD'] = but_low['sample_id'].map(OD_low)
#scale the measured value by the optical density as a proxy for flux per dry cell weight
but_low['measured'] = but_low['sample_id'].map(meas_but_low)/but_low['OD']
but_low

FileNotFoundError: [Errno 2] No such file or directory: '../models'

### Plot results for low richness communities

In [ ]:
plt = ( #plot scaled measured value vs predicted value for butyrate flux
    ggplot(
    but_low, aes(x = 'measured',y = 'predicted'))
    +geom_point(size = 3, color = 'cadetblue')
    +geom_smooth(method = 'lm', color = 'cadetblue',linetype =  '--')
    +labs(x = 'Measured Production Rate ($\dfrac{mmol}{h}$)', y = 'Predicted Production Rate ($\dfrac{mmol}{gDW*h}$)')
    +theme(text = element_text(size=15),panel_background=element_rect(fill = "white",
                                colour = "white",size = 0.5, linetype = "solid"),
                                panel_grid=element_blank(),
                                axis_line = element_line(size = 2, linetype = "solid",colour = "black"),
                                legend_title=element_blank(),
                                legend_position='right'))
plt

### Use Pearson correlation to test relationship between measured and predicted values

In [ ]:
rho, p = scipy.stats.pearsonr(but_low['measured'], but_low['predicted'])
p

## Now, we'll repeat this for high richness models, with 11-25 taxa per community

In [ ]:
#filter to high richness
high = main[(main['richness'].astype('int')>=10)] 
#get fractional abundances
taxa_high = [cols for cols in high.columns if 'Fraction' in cols] 
#drop b. cereus
taxa_high = high[taxa_high].drop(columns = ['B.cereus Fraction'])
#create dicts for optical density, butyrate
OD_high = high['OD'].to_dict()
meas_but_high = high['Butyrate'].to_dict()

### Build the models for high richness communities

In [ ]:
os.chdir('../agora/data')
#build the taxonomy table using the build_table function
taxonomy = build_table(taxa_high)
#build models with a cutoff of 0.001
manifest_high = micom.workflows.build(taxonomy, out_folder='../../models/invitro/high_richness',
                                  model_db = None, cutoff=0.001, threads=20)

### Grow the models for high richness communities

In [ ]:
os.chdir('../models')
#load the manifest for the high richness communities
manifest_high = pd.read_csv('invitro/high_richness/manifest.csv')
#fix the medium such that all samples can grow with a community growth rate of 0.3
medium_fixed = micom.workflows.fix_medium(manifest_low,'invitro/high_richness', medium, community_growth = 0.3, summarize = True)
#grow the models, using the medium and a tradeoff value of 0.7
growth = micom.workflows.grow(manifest_low,'invitro/high_richness',medium_fixed,
                           tradeoff = 0.7, threads = 10)
exchanges = micom.measures.production_rates(growth)
but_high = exchanges[exchanges.reaction.str.startswith('EX_but(e)')].rename(columns = {'flux':'predicted'})
but_high['OD'] = but_high['sample_id'].map(OD_high)
but_high['measured'] = but_high['sample_id'].map(meas_but_high)/but_high['OD']

### Plot High Richness Results

In [ ]:
plt = ( #plot scaled measured value vs predicted value for butyrate flux
    ggplot(
    but_high, aes(x = 'measured',y = 'predicted'))
    +geom_point(size = 3, color = "cadetblue")
    +geom_smooth(method = 'lm', color = "cadetblue", linetype = '--')
    +labs(x = 'Measured Production Rate ($\dfrac{mmol}{h}$)', y = 'Predicted Production Rate ($\dfrac{mmol}{gDW*h}$)')
    +theme(text = element_text(size=20),panel_background=element_rect(fill = "white",
                                colour = "white",size = 0.5, linetype = "solid"),panel_grid= element_blank(),
                                axis_line = element_line(size = 2, linetype = "solid",colour = "black"),
                                legend_title=element_blank(),
                                legend_position='right'))
plt

### Use Pearson correlation to test relationship between measured and predicted values

In [ ]:
#Check the pearson correlation statistics
rho, p = scipy.stats.pearsonr(but_high['measured'], but_high['predicted'])
p

## Save the resulting dataframes for low and high richness communities for use in building the final figures

In [ ]:
os.chdir('../results')
but_low.to_csv('invitro_low.csv')
but_high.to_csv('invitro_high.csv')